In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

warnings.filterwarnings('ignore')

In [ ]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
train

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
#train을 target과 feature로 나눠줍니다.
#train_ = train.dropna(axis=0)
train_= IterativeImputer().fit_transform(train.iloc[:, :-1])
train_= pd.DataFrame(train_)
test= IterativeImputer().fit_transform(test)
test= pd.DataFrame(test)
#train_ = train

In [ ]:
train_.info()

In [ ]:
norm=Normalizer()
sc=StandardScaler()
train_S = sc.fit_transform(train_)
train_S = pd.DataFrame(train_S)
test=sc.transform(test) 
test = pd.DataFrame(test)

In [ ]:
train_S.columns=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8'
               ,'Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16'
               ,'Q17','Q18','Q19','Q20','Q21','Q22','Q23','Q24'
               ,'Q25','Q26','introelapse','testelapse','surveyelapse'
                ,'TIPI1','TIPI2','TIPI3'
               ,'TIPI4','TIPI5','TIPI6','TIPI7','TIPI8','TIPI9','TIPI10'
                ,'VCL1','VCL2','VCL3','VCL4','VCL5','VCL6','VCL7'
               ,'VCL8','VCL9','VCL10','VCL11','VCL12','VCL13','VCL14'
               ,'VCL15','VCL16','education','urban','gender','engnat','age'
               ,'hand','religion','orientation','voted','married','familysize'
                ,'ASD']


In [ ]:
train_S

In [ ]:
train_S['nerdiness']=train['nerdiness']

In [ ]:
train_x = train_S.drop(['nerdiness'],axis=1)
train_y = train_S['nerdiness']

In [ ]:
start_time=time.time()
lgb= lgb.LGBMClassifier(
            n_estimators=1000
        )
parametes= {
    'num_leaves':[20,40,60,80,100],'min_child_samples':[5,10,15],
    'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]
}
clf=RandomizedSearchCV(lgb,parametes,scoring='roc_auc',n_iter=100)
clf.fit(X=train_x,y=train_y)
print(clf.best_params_)
end_time=time.time()
print('소요 시간 : ')
print(end_time-start_time)

In [ ]:
# 재학습 위한 공간

In [ ]:
#lgbm_clf.fit(train_x, train_y)
lgbm_pred = clf.predict(test)

In [ ]:
submission = pd.read_csv('competition_data/sample_submission.csv')

submission

In [ ]:
submission["nerdiness"] = lgbm_pred

In [ ]:
submission

In [ ]:
submission.to_csv("baseline_notNA-iterative-withSC-Random1000-test2.csv", index = False)